In [17]:
%config IPCompleter.use_jedi = False
%pdb off
%load_ext autoreload
%autoreload 3
import os
import queue
import threading
import numpy as np
import pandas as pd
import panel as pn
from pathlib import Path
import shutil # for _backup_extant_file(...)
import param

# pn.extension('vtk', design='material', sizing_mode='stretch_width', template='material')=
pn.extension('texteditor', 'plotly', 'terminal', template='material')

pn.state.template.config.raw_css.append("""
#main {
  padding: 0;
}""")

Automatic pdb calling has been turned OFF
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from pyphoplacecellanalysis.General.Batch.LogFileWatcher import LogFileHandler, start_monitoring, get_sorted_log_files

log_queue = queue.Queue()
path_to_watch = r"K:\scratch\gen_scripts" # Set the path to the directory containing log files


In [37]:
test_folders = ['K:/scratch/gen_scripts/run_kdiba_gor01_one_2006-6-08_14-26-15',
 'K:/scratch/gen_scripts/run_kdiba_gor01_one_2006-6-09_1-22-43',
 'K:/scratch/gen_scripts/run_kdiba_gor01_one_2006-6-12_15-55-31',
 'K:/scratch/gen_scripts/run_kdiba_gor01_two_2006-6-07_16-40-19',
 'K:/scratch/gen_scripts/run_kdiba_gor01_two_2006-6-08_21-16-25',
 'K:/scratch/gen_scripts/run_kdiba_gor01_two_2006-6-09_22-24-40',
 'K:/scratch/gen_scripts/run_kdiba_gor01_two_2006-6-12_16-53-46',
 'K:/scratch/gen_scripts/run_kdiba_vvp01_one_2006-4-09_17-29-30',
 'K:/scratch/gen_scripts/run_kdiba_vvp01_one_2006-4-10_12-25-50',
 'K:/scratch/gen_scripts/run_kdiba_vvp01_two_2006-4-09_16-40-54',
 'K:/scratch/gen_scripts/run_kdiba_vvp01_two_2006-4-10_12-58-3',
 'K:/scratch/gen_scripts/run_kdiba_pin01_one_11-02_17-46-44',
 'K:/scratch/gen_scripts/run_kdiba_pin01_one_11-02_19-28-0',
 'K:/scratch/gen_scripts/run_kdiba_pin01_one_11-03_12-3-25',
 'K:/scratch/gen_scripts/run_kdiba_pin01_one_fet11-01_12-58-54']

test_folders = [Path(v).resolve() for v in test_folders]

In [9]:

test_files = [r"K:\scratch\gen_scripts\run_kdiba_gor01_one_2006-6-08_14-26-15\debug_2024-05-28_03-05-55.Apogee.kdiba.gor01.one.2006-6-08_14-26-15.log",
            r"K:\scratch\gen_scripts\run_kdiba_gor01_two_2006-6-08_21-16-25\debug_2024-05-28_03-05-28.Apogee.kdiba.gor01.two.2006-6-08_21-16-25.log",
            r"K:\scratch\gen_scripts\run_kdiba_gor01_two_2006-6-07_16-40-19\debug_2024-05-28_03-05-24.Apogee.kdiba.gor01.two.2006-6-07_16-40-19.log",
            r"K:\scratch\gen_scripts\run_kdiba_gor01_two_2006-6-09_22-24-40\debug_2024-05-28_04-05-38.Apogee.kdiba.gor01.two.2006-6-09_22-24-40.log",
            ]

for a_file in test_files:
    with open(a_file, 'r') as file:
        contents = file.read()
        log_queue.put((a_file, contents))


In [10]:
# Path(path_to_watch).resolve().iterdir()

parent_dirs = [Path(a_file).resolve().parent for a_file in test_files]
parent_dirs

[WindowsPath('K:/scratch/gen_scripts/run_kdiba_gor01_one_2006-6-08_14-26-15'),
 WindowsPath('K:/scratch/gen_scripts/run_kdiba_gor01_two_2006-6-08_21-16-25'),
 WindowsPath('K:/scratch/gen_scripts/run_kdiba_gor01_two_2006-6-07_16-40-19'),
 WindowsPath('K:/scratch/gen_scripts/run_kdiba_gor01_two_2006-6-09_22-24-40')]

In [44]:
_out_sorted_log_files_dict = {}

for a_dir in test_folders:
    # a_dir = Path(a_dir).resolve()
    print(f'for "{a_dir}"')
    backup_dir = a_dir.joinpath('EXTERNAL', 'BAK').resolve()
    backup_dir.mkdir(exist_ok=True, parents=True)
    # print(backup_dir)
    ## move all old log files into the backup_dir
    # curr_log_files = get_sorted_log_files(str(a_dir.absolute()), recursive=True)
    curr_log_files = get_sorted_log_files(str(a_dir), recursive=False)
    print(f'\tcurr_log_files: {curr_log_files}')
    _out_sorted_log_files_dict[a_dir.name] = curr_log_files

    for log_file in _out_sorted_log_files_dict[a_dir.name]:
        log_file = Path(log_file).resolve()
        print(f'\tlog_file: "{log_file}"')
        if log_file.parent.name not in ['BAK', 'log_file.log', 'OLD', 'EXTERNAL']:
            backup_log_file_location = backup_dir.joinpath(log_file.relative_to(a_dir)).resolve()
            # print(f'\t\tmoving "{log_file.relative_to(a_dir)}" to "{backup_log_file_location.relative_to(a_dir)}"...')
            print(f'\t\tmoving "{log_file}" to "{backup_log_file_location}"...')
            if (log_file.exists() and log_file.is_file()):
                shutil.move(str(log_file.absolute()), str(backup_log_file_location.absolute()))
                
        #     # with open(log_file, 'r') as file:
        #     #     contents = file.read()
        #     #     log_queue.put((log_file, contents))

for "K:\scratch\gen_scripts\run_kdiba_gor01_one_2006-6-08_14-26-15"
	curr_log_files: ['K:\\scratch\\gen_scripts\\run_kdiba_gor01_one_2006-6-08_14-26-15\\debug_2024-04-28_01-04-05.Apogee.kdiba.gor01.one.2006-6-08_14-26-15.log', 'K:\\scratch\\gen_scripts\\run_kdiba_gor01_one_2006-6-08_14-26-15\\debug_2024-04-28_01-04-00.Apogee.kdiba.gor01.one.2006-6-08_14-26-15.log', 'K:\\scratch\\gen_scripts\\run_kdiba_gor01_one_2006-6-08_14-26-15\\debug_2024-04-28_01-04-25.Apogee.kdiba.gor01.one.2006-6-08_14-26-15.log', 'K:\\scratch\\gen_scripts\\run_kdiba_gor01_one_2006-6-08_14-26-15\\debug_2024-04-29_11-04-00.Apogee.kdiba.gor01.one.2006-6-08_14-26-15.log', 'K:\\scratch\\gen_scripts\\run_kdiba_gor01_one_2006-6-08_14-26-15\\debug_2024-05-07_22-05-35.Apogee.kdiba.gor01.one.2006-6-08_14-26-15.log', 'K:\\scratch\\gen_scripts\\run_kdiba_gor01_one_2006-6-08_14-26-15\\debug_2024-05-08_09-05-38.Apogee.kdiba.gor01.one.2006-6-08_14-26-15.log', 'K:\\scratch\\gen_scripts\\run_kdiba_gor01_one_2006-6-08_14-26-15\\d

In [13]:
from pyphoplacecellanalysis.General.Batch.LogFileWatcher import get_sorted_log_files


all_sessions_sorted_log_files = get_sorted_log_files(path_to_watch)


_out_sorted_log_files_dict = {}

for a_dir in parent_dirs:
    _out_sorted_log_files_dict[a_dir.name] = get_sorted_log_files(a_dir)
    for log_file in _out_sorted_log_files_dict[a_dir.name]:
        print(log_file)
        # with open(log_file, 'r') as file:
        #     contents = file.read()
        #     log_queue.put((log_file, contents))


K:\scratch\gen_scripts\run_kdiba_gor01_one_2006-6-08_14-26-15\debug_com.PhoHale.PhoPy3DPositionAnalyis.Batch.runBatch.run_specific_batch.Apogee.kdiba.gor01.one.2006-6-08_14-26-15.log
K:\scratch\gen_scripts\run_kdiba_gor01_one_2006-6-08_14-26-15\EXTERNAL\TESTING\Logging\debug_com.PhoHale.PhoPy3DPositionAnalyis.Batch.runBatch.run_specific_batch.Apogee.kdiba.gor01.one.2006-6-08_14-26-15.log
K:\scratch\gen_scripts\run_kdiba_gor01_one_2006-6-08_14-26-15\EXTERNAL\TESTING\Logging\debug_com.PhoHale.Spike3D.pipeline.log
K:\scratch\gen_scripts\run_kdiba_gor01_one_2006-6-08_14-26-15\debug_2024-04-28_01-04-05.Apogee.kdiba.gor01.one.2006-6-08_14-26-15.log
K:\scratch\gen_scripts\run_kdiba_gor01_one_2006-6-08_14-26-15\debug_2024-04-28_01-04-00.Apogee.kdiba.gor01.one.2006-6-08_14-26-15.log
K:\scratch\gen_scripts\run_kdiba_gor01_one_2006-6-08_14-26-15\debug_2024-04-28_01-04-25.Apogee.kdiba.gor01.one.2006-6-08_14-26-15.log
K:\scratch\gen_scripts\run_kdiba_gor01_one_2006-6-08_14-26-15\debug_2024-04-29_11

In [14]:
_out_sorted_log_files_dict[a_dir.name]

['K:\\scratch\\gen_scripts\\run_kdiba_gor01_two_2006-6-09_22-24-40\\debug_com.PhoHale.PhoPy3DPositionAnalyis.Batch.runBatch.run_specific_batch.Apogee.kdiba.gor01.two.2006-6-09_22-24-40.log',
 'K:\\scratch\\gen_scripts\\run_kdiba_gor01_two_2006-6-09_22-24-40\\log_file.log',
 'K:\\scratch\\gen_scripts\\run_kdiba_gor01_two_2006-6-09_22-24-40\\EXTERNAL\\TESTING\\Logging\\debug_com.PhoHale.Spike3D.pipeline.log',
 'K:\\scratch\\gen_scripts\\run_kdiba_gor01_two_2006-6-09_22-24-40\\log_file.log\\debug_com.PhoHale.PhoPy3DPositionAnalyis.Batch.runBatch.run_specific_batch.Apogee.kdiba.gor01.two.2006-6-09_22-24-40.log',
 'K:\\scratch\\gen_scripts\\run_kdiba_gor01_two_2006-6-09_22-24-40\\debug_2024-04-28_01-04-40.Apogee.kdiba.gor01.two.2006-6-09_22-24-40.log',
 'K:\\scratch\\gen_scripts\\run_kdiba_gor01_two_2006-6-09_22-24-40\\debug_2024-04-28_01-04-11.Apogee.kdiba.gor01.two.2006-6-09_22-24-40.log',
 'K:\\scratch\\gen_scripts\\run_kdiba_gor01_two_2006-6-09_22-24-40\\debug_2024-04-28_01-04-55.Apogee

In [ ]:
wysiwyg = pn.widgets.TextEditor(placeholder='Enter some text')
wysiwyg

In [ ]:
terminal = pn.widgets.Terminal(
    "Welcome to the Panel Terminal!\nI'm based on xterm.js\n\n",
    options={"cursorBlink": True},
    height=300, sizing_mode='stretch_width'
)

terminal.show()

pn.pane.

In [ ]:
subprocess_terminal = pn.widgets.Terminal(
    options={"cursorBlink": True},
    height=300, sizing_mode='stretch_width'
)

run_python = pn.widgets.Button(name="Run Python", button_type="success")
run_python.on_click(
    lambda x: subprocess_terminal.subprocess.run("python")
)

kill = pn.widgets.Button(name="Kill Python", button_type="danger")
kill.on_click(
    lambda x: subprocess_terminal.subprocess.kill()
)

pn.Column(
    pn.Row(run_python, kill, subprocess_terminal.subprocess.param.running),
    subprocess_terminal,
    sizing_mode='stretch_both',
    min_height=500
)

In [ ]:
import plotly.graph_objs as go

xx = np.linspace(-3.5, 3.5, 100)
yy = np.linspace(-3.5, 3.5, 100)
x, y = np.meshgrid(xx, yy)
z = np.exp(-(x-1)**2-y**2)-(x**3+y**4-x/5)*np.exp(-(x**2+y**2))

surface = go.Surface(z=z)
layout = go.Layout(
    title='Plotly 3D Plot',
    autosize=True,
    margin=dict(t=50, b=50, r=50, l=50)
)
fig = dict(data=[surface], layout=layout)

plotly_pane = pn.pane.Plotly(fig)

plotly_pane



In [ ]:
import numpy as np
import matplotlib

matplotlib.use('agg')

import matplotlib.pyplot as plt

Y, X = np.mgrid[-3:3:100j, -3:3:100j]
U = -1 - X**2 + Y
V = 1 + X - Y**2
speed = np.sqrt(U*U + V*V)

fig0, ax0 = plt.subplots()
strm = ax0.streamplot(X, Y, U, V, color=U, linewidth=2, cmap=plt.cm.autumn)
fig0.colorbar(strm.lines)

mpl_pane = pn.pane.Matplotlib(fig0, format='svg', sizing_mode='stretch_both')

mpl_pane

In [ ]:

pn.Tabs(
    # ('Altair', altair_pane),
    # ('Bokeh', bokeh_pane),
    # ('deck.GL', deck_gl),
    # ('Echarts', echarts_pane),
    # ('HoloViews', hv_panel),
    ('Matplotlib', mpl_pane),
    ('Plotly', plotly_pane),
    dynamic=True, sizing_mode='stretch_both'
).servable()

In [ ]:
# Creation of a mapping between Custom name and the VTK object reference
actor_ref = ["None", actor.__this__]
actor_names = ["None", 'St Helen']
actor_opts = {k:v for k,v in zip(actor_names, actor_ref)}

options = {}
actor_selection = pn.widgets.Select(value=None, options=actor_opts, margin=0, name="Actor Selection")
actor_selection